In [ ]:
import cv2
import numpy as np # Moved here from later in the cell, and ensured no double import.

# !pip install torch torchvision transformers datasets accelerate evaluate
# !pip install opencv-python pillow matplotlib

from transformers import Mask2FormerForUniversalSegmentation
from transformers import AutoImageProcessor
import torch

model_name = "facebook/mask2former-swin-base-ade-semantic"

processor = AutoImageProcessor.from_pretrained(model_name)
model = Mask2FormerForUniversalSegmentation.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

from google.colab import files
print("Upload ROOM image:")
uploaded = files.upload()

from PIL import Image
# import numpy as np # Already imported at the top

image = Image.open(list(uploaded.keys())[0]).convert("RGB")
image

inputs = processor(images=image, return_tensors="pt").to(device)

with torch.no_grad():
    outputs = model(**inputs)

result = processor.post_process_semantic_segmentation(
    outputs,
    target_sizes=[image.size[::-1]]
)[0]

segmentation_map = result.cpu().numpy()

model.config.id2label
WALL_ID = 0
FLOOR_ID = 3

wall_mask = (segmentation_map == WALL_ID).astype(np.uint8) * 255
floor_mask = (segmentation_map == FLOOR_ID).astype(np.uint8) * 255

import matplotlib.pyplot as plt

plt.figure(figsize=(15,5))

plt.subplot(1,3,1)
plt.imshow(image)
plt.title("Original")

plt.subplot(1,3,2)
plt.imshow(wall_mask, cmap="gray")
plt.title("Wall Mask")

plt.subplot(1,3,3)
plt.imshow(floor_mask, cmap="gray")
plt.title("Floor Mask")

plt.show()

from google.colab import files
print("Upload Tile image:")
uploaded_tile = files.upload()

tile_img = Image.open(list(uploaded_tile.keys())[0]).convert("RGB")
tile_img

# import numpy as np # Already imported at the top

tile_np = np.array(tile_img)
room_h, room_w = image.size[1], image.size[0]

tile_h, tile_w = tile_np.shape[:2]

repeat_y = room_h // tile_h + 1
repeat_x = room_w // tile_w + 1

big_tile = np.tile(tile_np, (repeat_y, repeat_x, 1))
big_tile = big_tile[:room_h, :room_w]

room_np = np.array(image)

floor_mask_bool = floor_mask.astype(bool)

result_img = room_np.copy()
result_img[floor_mask_bool] = big_tile[floor_mask_bool]

plt.figure(figsize=(10,8))
plt.imshow(result_img)
plt.axis("off")
plt.title("Tile Applied to Floor")
plt.show()

# import cv2 # Already imported at the top
# import numpy as np # Already imported at the top

def create_tile_pattern(width, height, tile_img, tile_size=120, grout=3):

    grout_color = 220
    pattern = np.ones((height, width, 3), dtype=np.uint8) * grout_color

    tile_resized = cv2.resize(tile_img, (tile_size-grout, tile_size-grout))

    for y in range(0, height, tile_size):
        for x in range(0, width, tile_size):

            variation = 0.95 + 0.1*np.random.rand()
            tile_var = np.clip(tile_resized * variation, 0, 255).astype(np.uint8)

            y_end = min(y+tile_size-grout, height)
            x_end = min(x+tile_size-grout, width)

            pattern[y:y_end, x:x_end] = tile_var[:y_end-y, :x_end-x]

    return pattern

def order_points_clockwise(pts):

    rect = np.zeros((4,2), dtype="float32")

    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]  # top-left
    rect[2] = pts[np.argmax(s)]  # bottom-right

    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]  # top-right
    rect[3] = pts[np.argmax(diff)]  # bottom-left

    return rect
def get_floor_corners(mask):

    mask = (mask > 0).astype(np.uint8) * 255

    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnt = max(contours, key=cv2.contourArea)

    # Convex hull (important!)
    hull = cv2.convexHull(cnt)

    epsilon = 0.02 * cv2.arcLength(hull, True)
    approx = cv2.approxPolyDP(hull, epsilon, True)

    if len(approx) < 4:
        return None

    pts = approx.reshape(-1, 2)

    # If more than 4 points, take extreme 4 via bounding quad
    if len(pts) > 4:
        rect = cv2.minAreaRect(hull)
        pts = cv2.boxPoints(rect)

    pts = order_points_clockwise(pts)

    return np.float32(pts)
def extract_lighting(original_img):
    gray = cv2.cvtColor(original_img, cv2.COLOR_BGR2GRAY)
    gray_blur = cv2.GaussianBlur(gray, (51,51), 0)
    lighting = gray_blur / 255.0
    return lighting

# Removed redundant and mis-indented imports of numpy and cv2 from here.

# Convert PIL to OpenCV format
image = np.array(image)

# Convert RGB (PIL) → BGR (OpenCV)
image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

# Convert PIL Image to numpy array (RGB)
tile_img_np = np.array(tile_img)
# Convert RGB (numpy) to BGR (OpenCV)
tile_img_cv = cv2.cvtColor(tile_img_np, cv2.COLOR_RGB2BGR)

def apply_realistic_tile(original_img, mask, tile_img, rotation_angle=0):
    H, W = original_img.shape[:2]
    mask = (mask > 0).astype(np.uint8) * 255

    # Get floor corners from your existing function
    dst_pts = get_floor_corners(mask)

    if dst_pts is None:
        print("Could not detect proper floor corners")
        return original_img

    # --- OPTION 2: BETTER SOURCE POINT MAPPING ---
    # We create a much larger source plane to ensure we aren't "zooming in"
    # too much on a small texture, which causes blurriness.
    tex_scale = 800  # Decreased to make tiles appear larger/more visible
    src_pts = np.float32([
        [0, 0],
        [tex_scale, 0],
        [tex_scale, tex_scale],
        [0, tex_scale]
    ])

    # Create the pattern based on the tex_scale
    tile_pattern = create_tile_pattern(tex_scale, tex_scale, tile_img, tile_size=100) # Adjusted tile_size for visibility

    # Apply Rotation to the source points
    center = (tex_scale / 2, tex_scale / 2)
    rot_mat = cv2.getRotationMatrix2D(center, rotation_angle, 1.0)

    # Expand src_pts to (4, 1, 2) for cv2.transform
    rotated_src_pts = cv2.transform(src_pts.reshape(-1, 1, 2), rot_mat).reshape(-1, 2)

    # Compute homography and warp
    H_matrix = cv2.getPerspectiveTransform(rotated_src_pts, dst_pts)
    warped_tiles = cv2.warpPerspective(tile_pattern, H_matrix, (W, H), flags=cv2.INTER_LANCZOS4)

    # --- OPTION 3: REFINED LIGHTING & BLENDING ---
    # Extract lighting but prevent it from turning the tiles black/gray
    gray = cv2.cvtColor(original_img, cv2.COLOR_BGR2GRAY)

    # Smaller kernel (15,15) preserves contact shadows (occlusion) better than (31,31)
    lighting = cv2.GaussianBlur(gray, (15, 15), 0).astype(np.float32) / 255.0

    # Lift the shadows: ensures tiles in dark areas are still visible
    # Formula: 0.4 (base brightness) + 0.6 (room lighting)
    lighting = 0.4 + (0.6 * lighting)

    # Apply lighting to the warped texture
    realistic_tiles = warped_tiles.astype(np.float32) * lighting[:, :, None]
    realistic_tiles = np.clip(realistic_tiles, 0, 255).astype(np.uint8)

    # Sharp mask blending
    # Using a slight blur on the mask edge (3,3) prevents jagged "pixel" edges
    # without making the floor look like it's glowing.
    mask_blur = cv2.GaussianBlur(mask, (3, 3), 0) / 255.0
    mask_3 = np.stack([mask_blur] * 3, axis=-1)

    # Final Composite
    result = (original_img.astype(np.float32) * (1 - mask_3) + 
              realistic_tiles.astype(np.float32) * mask_3).astype(np.uint8)

    return result
result_rotated = apply_realistic_tile(image, floor_mask, tile_img_cv, rotation_angle=20)
#result_rotated = apply_realistic_tile(image, wall_mask, tile_img_cv, rotation_angle=-10)

plt.figure(figsize=(12,8))
plt.imshow(cv2.cvtColor(result_rotated, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.title("Tile Applied to Floor")
plt.show()